In [1]:
import sys
sys.path.append("../set-generation")
import csv
import json
import numpy as np
import numba
import color_conversions

In [2]:
ALL_NUM_COLORS = [6, 8, 10]

In [3]:
def to_rgb(color):
    """
    Convert hex color code (without `#`) to sRGB255.
    """
    return np.array([(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in color])

In [4]:
@numba.njit
def calc_min_dists(rgb):
    """Calculate min delta E for each set."""
    nc = rgb.shape[1]
    min_dists = []
    rgb_linear = color_conversions.sRGB1_to_sRGB1_linear(rgb.flatten()).reshape(
        rgb.shape
    )
    for c in range(rgb_linear.shape[0]):
        min_dist = 100
        for i in range(1, nc):
            for severity in range(1, 101):
                jab1 = color_conversions.rgb_linear_to_jab(rgb_linear[c, i])
                deut1 = color_conversions.rgb_linear_to_jab(
                    color_conversions.CVD_forward_deuteranomaly(
                        rgb_linear[c, i], severity
                    )
                )
                prot1 = color_conversions.rgb_linear_to_jab(
                    color_conversions.CVD_forward_protanomaly(
                        rgb_linear[c, i], severity
                    )
                )
                trit1 = color_conversions.rgb_linear_to_jab(
                    color_conversions.CVD_forward_tritanomaly(
                        rgb_linear[c, i], severity
                    )
                )
                for j in range(i):
                    jab2 = color_conversions.rgb_linear_to_jab(rgb_linear[c, j])
                    deut2 = color_conversions.rgb_linear_to_jab(
                        color_conversions.CVD_forward_deuteranomaly(
                            rgb_linear[c, j], severity
                        )
                    )
                    prot2 = color_conversions.rgb_linear_to_jab(
                        color_conversions.CVD_forward_protanomaly(
                            rgb_linear[c, j], severity
                        )
                    )
                    trit2 = color_conversions.rgb_linear_to_jab(
                        color_conversions.CVD_forward_tritanomaly(
                            rgb_linear[c, j], severity
                        )
                    )
                    min_dist = min(min_dist, color_conversions.cam02de(jab1, jab2))
                    min_dist = min(min_dist, color_conversions.cam02de(deut1, deut2))
                    min_dist = min(min_dist, color_conversions.cam02de(prot1, prot2))
                    min_dist = min(min_dist, color_conversions.cam02de(trit1, trit2))
        min_dists.append(min_dist)
    return min_dists

## With original lightness constraints

In [5]:
COLOR_FILE = {
    6: "../survey/color-sets/colors_mcd20_mld2_nc6_cvd100_minj40_maxj90_ns10000.txt",
    8: "../survey/color-sets/colors_mcd18_mld2_nc8_cvd100_minj40_maxj90_ns10000.txt",
    10: "../survey/color-sets/colors_mcd16_mld2_nc10_cvd100_minj40_maxj90_ns10000.txt",
}

In [6]:
# Load color data
colors_rgb = {}
for num_colors in ALL_NUM_COLORS:
    with open(COLOR_FILE[num_colors]) as csv_file:
        # Skip header rows
        csv_file.readline()
        csv_file.readline()
        csv_file.readline()
        csv_reader = csv.reader(csv_file, delimiter=" ")
        colors_hex = np.array([[i.strip() for i in row] for row in csv_reader])
        colors_rgb[num_colors] = np.array([to_rgb(i) for i in colors_hex]) / 255

In [7]:
min_dists = {nc: np.array(calc_min_dists(colors_rgb[nc])) for nc in ALL_NUM_COLORS}

In [8]:
print("maximum minimum-color distances:")
for nc in ALL_NUM_COLORS:
    print(f"{nc:2d}: {np.max(min_dists[nc]):.1f} [{np.argmax(min_dists[nc]):04d}]")

maximum minimum-color distances:
 6: 24.3 [9764]
 8: 20.2 [1811]
10: 18.0 [6391]


In [9]:
print("mean minimum-color distances:")
for nc in ALL_NUM_COLORS:
    print(f"{nc:2d}: {np.mean(min_dists[nc]):.1f}")

mean minimum-color distances:
 6: 20.5
 8: 18.2
10: 16.1


## With tighter lightness constraints

In [10]:
COLOR_FILE = {
    6: "../set-generation/colors_mcd20.0_mld5.0_nc6_cvd100_minj40_maxj80_ns10000_f.txt",
    8: "../set-generation/colors_mcd18.0_mld4.2_nc8_cvd100_minj40_maxj82_ns10000_f.txt",
    10: "../set-generation/colors_mcd16.0_mld3.6_nc10_cvd100_minj40_maxj84_ns10000_f.txt",
}

In [11]:
# Load color data
colors_rgb = {}
colors_hex = {}
for num_colors in ALL_NUM_COLORS:
    with open(COLOR_FILE[num_colors]) as csv_file:
        # Skip header rows
        csv_file.readline()
        csv_file.readline()
        csv_file.readline()
        csv_reader = csv.reader(csv_file, delimiter=" ")
        colors_hex[num_colors] = np.array(
            [[i.strip() for i in row] for row in csv_reader]
        )
        colors_rgb[num_colors] = (
            np.array([to_rgb(i) for i in colors_hex[num_colors]]) / 255
        )

In [12]:
min_dists = {nc: np.array(calc_min_dists(colors_rgb[nc])) for nc in ALL_NUM_COLORS}

In [13]:
print("maximum minimum-color distances:")
max_min_dist_sets = {}
for nc in ALL_NUM_COLORS:
    max_min_dist_sets[nc] = list(colors_hex[nc][np.argmax(min_dists[nc])])
    print(f"{nc:2d}: {np.max(min_dists[nc]):.1f} [{np.argmax(min_dists[nc]):04d}]")

maximum minimum-color distances:
 6: 23.6 [5793]
 8: 19.6 [9802]
10: 16.9 [4964]


In [14]:
print("mean minimum-color distances:")
for nc in ALL_NUM_COLORS:
    print(f"{nc:2d}: {np.mean(min_dists[nc]):.1f}")

mean minimum-color distances:
 6: 20.3
 8: 18.1
10: 16.1


In [15]:
with open("max-min-dist-sets.json", "w") as outfile:
    json.dump(max_min_dist_sets, outfile)